# 10 Fold CV

In [1]:
import pandas as pd
from Bio import SeqIO
import os
import random
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Input, Flatten, LSTM, Dropout, Bidirectional, LeakyReLU, Reshape, Lambda
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

# performance matrices
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score

# plots
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from numpy import array
from numpy import argmax

from tensorflow.keras.regularizers import l1, l2


from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler

import imblearn
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc
import os 
import pandas as pd
import numpy as np

In [2]:
os.chdir("/home/t326h379/OGP")


df_negative = pd.read_csv('Feature_Extraction_O_linked_Training_Negative_114307_Sites_less.txt',header=None)

df_positive = pd.read_csv('Feature_Extraction_O_linked_Training_Positive_4885_Sites_less.txt',header=None)

Header_name = ["Position","PID","Position_redundant","81 Window sequence","S or T"]

col_of_feature = [i for i in range(1,1025)]

Header_name = Header_name + col_of_feature

df_positive.columns = Header_name
df_negative.columns = Header_name


frames = [df_positive, df_negative]

O_linked_training = pd.concat(frames,ignore_index = True)

df_Train_array = O_linked_training.drop(["Position","PID","Position_redundant","81 Window sequence","S or T"],axis=1)
df_Train_array = np.array(df_Train_array)

X_train_full = df_Train_array

y_train_full = np.array([1]*4885+[0]*114144)

In [3]:
X_train_full.shape

(119029, 1024)

In [4]:
y_train_full.shape

(119029,)

In [5]:
print("Prior")
print(X_train_full.shape)
print(y_train_full.shape)
seed = 42
rus = RandomUnderSampler(random_state = seed)
X_train_full, y_train_full = rus.fit_resample(X_train_full,y_train_full)
print("After")
print(X_train_full.shape)
print(y_train_full.shape)

Prior
(119029, 1024)
(119029,)
After
(9770, 1024)
(9770,)


In [6]:
X_train_full = X_train_full.reshape(X_train_full.shape[0],1024,1)

In [7]:
X_train_full.shape

(9770, 1024, 1)

In [8]:
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

In [9]:
X_train_full, y_train_full = shuffle(X_train_full, y_train_full)

print(X_train_full.shape)
print(y_train_full.shape)


MCC = []
kfold = StratifiedKFold(n_splits=10)
cvscores, auc_scores, sn, sp, acc = list(), list(), list(), list(), list()

for train_index, test_index in kfold.split(X_train_full, y_train_full):
    xtrain, xval = X_train_full[train_index], X_train_full[test_index]
    ytrain, yval = y_train_full[train_index], y_train_full[test_index]      

    X_train_10= xtrain
    Y_train_10=ytrain

    Y_train_10 = tf.keras.utils.to_categorical(Y_train_10,2)
    
    model = Sequential()

    model.add(Input(shape=(1024,1)))

    model.add(Conv1D(filters=64,kernel_size=3,activation='relu',name='Conv_1D_1_add'))
    model.add(MaxPooling1D(pool_size=2,name="MaxPooling1D"))
    model.add(Dropout(0.3))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(32,activation='relu',name="Dense_1"))
    model.add(Dropout(0.3))

    model.add(Dense(2,activation='softmax',name="Dense_2"))

        
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),loss="binary_crossentropy",metrics=["accuracy"])

    checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath="ROC_ROC_Premise_Assumption.h5", 
                                    monitor = 'val_accuracy',
                                    verbose=0, 
                                    save_weights_only=False,
                                    save_best_only=True)

    reduce_lr_acc = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.001, patience=3, verbose=1, min_delta=1e-4, mode='max')

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=5,mode='max')

    history = model.fit(X_train_10, Y_train_10,epochs=35,verbose=1,batch_size=256)

    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.show()
    plt.savefig('accuracy_loss@@@@@@@@@@@@@@@_curve.png', dpi=300, bbox_inches='tight')

    Y_pred = model.predict(xval)
    Y_pred = (Y_pred > 0.5)
    y_pred = [np.argmax(y, axis=None, out=None) for y in Y_pred]
    y_pred = np.array(y_pred)
    print("Matthews Correlation : ",matthews_corrcoef(yval, y_pred))
    print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
    print("Accuracy on test set:   ",accuracy_score(yval, y_pred))

    cm = confusion_matrix(yval, y_pred)

    TP = cm[1][1]
    TN = cm[0][0]
    FP = cm[0][1]
    FN = cm[1][0]

    mcc = matthews_corrcoef(yval, y_pred)
    
    acc.append(accuracy_score(yval, y_pred))
    
    cvscores.append(mcc)

    Sensitivity = TP/(TP+FN)

    Specificity = TN/(TN+FP)
    
    sn.append(Sensitivity)
    sp.append(Specificity)

    print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


    fpr, tpr, _ = roc_curve(yval, y_pred)

    roc_auc_test = auc(fpr,tpr)



    print("Area Under Curve:   ",roc_auc_test)
    auc_scores.append(roc_auc_test)   


    model.summary()
    
    


print("Mean MCC = %.4f + %.4f and Mean AUC = %.4f + %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))
MCC.append(np.mean(cvscores))

print("Mean MCC:   ",np.mean(MCC))
print("Mean Sensitivity = %.4f + %.4f and Mean Specificity = %.4f + %.4f and Mean Accuracy = %.4f + %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))

print("\n\n\n\n\n\n\n\n\n\n\n\n\n")

(9770, 1024, 1)
(9770,)
Epoch 1/35
35/35 [==============================] - 0s 7ms/step - loss: 0.6668 - accuracy: 0.6231
Epoch 2/35
35/35 [==============================] - 0s 7ms/step - loss: 0.6095 - accuracy: 0.6963
Epoch 3/35
35/35 [==============================] - 0s 7ms/step - loss: 0.5710 - accuracy: 0.7214
Epoch 4/35
35/35 [==============================] - 0s 7ms/step - loss: 0.5513 - accuracy: 0.7381
Epoch 5/35
35/35 [==============================] - 0s 7ms/step - loss: 0.5361 - accuracy: 0.7518
Epoch 6/35
35/35 [==============================] - 0s 7ms/step - loss: 0.5227 - accuracy: 0.7656
Epoch 7/35
35/35 [==============================] - 0s 7ms/step - loss: 0.5161 - accuracy: 0.7724
Epoch 8/35
35/35 [==============================] - 0s 7ms/step - loss: 0.5057 - accuracy: 0.7770
Epoch 9/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4987 - accuracy: 0.7787
Epoch 10/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4912 - accuracy: 0.

35/35 [==============================] - 0s 6ms/step - loss: 0.4113 - accuracy: 0.8311
Epoch 31/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4099 - accuracy: 0.8326
Epoch 32/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4086 - accuracy: 0.8311
Epoch 33/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4039 - accuracy: 0.8333
Epoch 34/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4012 - accuracy: 0.8346
Epoch 35/35
35/35 [==============================] - 0s 6ms/step - loss: 0.4037 - accuracy: 0.8341
Matthews Correlation :  0.6625877874344758
Confusion Matrix : 
 [[438  50]
 [118 371]]
Accuracy on test set:    0.8280450358239508
Sensitivity:    0.7586912065439673 	 Specificity:    0.8975409836065574
Area Under Curve:    0.8281160950752623
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv_1D_1_add (Conv1D

35/35 [==============================] - 0s 6ms/step - loss: 0.5079 - accuracy: 0.7712
Epoch 7/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4949 - accuracy: 0.7837
Epoch 8/35
35/35 [==============================] - 0s 6ms/step - loss: 0.4888 - accuracy: 0.7871
Epoch 9/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4805 - accuracy: 0.7903
Epoch 10/35
35/35 [==============================] - 0s 6ms/step - loss: 0.4762 - accuracy: 0.7939
Epoch 11/35
35/35 [==============================] - 0s 6ms/step - loss: 0.4718 - accuracy: 0.7976
Epoch 12/35
35/35 [==============================] - 0s 6ms/step - loss: 0.4621 - accuracy: 0.8059
Epoch 13/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4585 - accuracy: 0.8029
Epoch 14/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4548 - accuracy: 0.8078
Epoch 15/35
35/35 [==============================] - 0s 6ms/step - loss: 0.4499 - accuracy: 0.8083
Epoch 16/35
35/35 [======

35/35 [==============================] - 0s 7ms/step - loss: 0.3944 - accuracy: 0.8387
Matthews Correlation :  0.6306121841036593
Confusion Matrix : 
 [[412  76]
 [105 384]]
Accuracy on test set:    0.8147389969293757
Sensitivity:    0.7852760736196319 	 Specificity:    0.8442622950819673
Area Under Curve:    0.8147691843507995
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv_1D_1_add (Conv1D)       (None, 1022, 64)          256       
_________________________________________________________________
MaxPooling1D (MaxPooling1D)  (None, 511, 64)           0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 511, 64)           0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 32704)             0         
______________________________________________________

35/35 [==============================] - 0s 7ms/step - loss: 0.4740 - accuracy: 0.7951
Epoch 12/35
35/35 [==============================] - 0s 6ms/step - loss: 0.4683 - accuracy: 0.7967
Epoch 13/35
35/35 [==============================] - 0s 6ms/step - loss: 0.4607 - accuracy: 0.8039
Epoch 14/35
35/35 [==============================] - 0s 6ms/step - loss: 0.4559 - accuracy: 0.8035
Epoch 15/35
35/35 [==============================] - 0s 6ms/step - loss: 0.4567 - accuracy: 0.8064
Epoch 16/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4487 - accuracy: 0.8131
Epoch 17/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4447 - accuracy: 0.8086
Epoch 18/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4411 - accuracy: 0.8156
Epoch 19/35
35/35 [==============================] - 0s 6ms/step - loss: 0.4379 - accuracy: 0.8118
Epoch 20/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4391 - accuracy: 0.8122
Epoch 21/35
35/35 [===

Epoch 1/35
35/35 [==============================] - 0s 7ms/step - loss: 0.6588 - accuracy: 0.6228
Epoch 2/35
35/35 [==============================] - 0s 7ms/step - loss: 0.5945 - accuracy: 0.7095
Epoch 3/35
35/35 [==============================] - 0s 6ms/step - loss: 0.5623 - accuracy: 0.7306
Epoch 4/35
35/35 [==============================] - 0s 6ms/step - loss: 0.5376 - accuracy: 0.7478
Epoch 5/35
35/35 [==============================] - 0s 7ms/step - loss: 0.5256 - accuracy: 0.7604
Epoch 6/35
35/35 [==============================] - 0s 7ms/step - loss: 0.5115 - accuracy: 0.7666
Epoch 7/35
35/35 [==============================] - 0s 7ms/step - loss: 0.5076 - accuracy: 0.7718
Epoch 8/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4985 - accuracy: 0.7805
Epoch 9/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4895 - accuracy: 0.7816
Epoch 10/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4839 - accuracy: 0.7906
Epoch 11/35
35/35 [

35/35 [==============================] - 0s 7ms/step - loss: 0.4164 - accuracy: 0.8267
Epoch 31/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4145 - accuracy: 0.8282
Epoch 32/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4164 - accuracy: 0.8260
Epoch 33/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4128 - accuracy: 0.8268
Epoch 34/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4124 - accuracy: 0.8257
Epoch 35/35
35/35 [==============================] - 0s 7ms/step - loss: 0.4091 - accuracy: 0.8308
Matthews Correlation :  0.6705967236304332
Confusion Matrix : 
 [[428  61]
 [101 387]]
Accuracy on test set:    0.834186284544524
Sensitivity:    0.7930327868852459 	 Specificity:    0.8752556237218814
Area Under Curve:    0.8341442053035637
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv_1D_1_add (Conv1D)

# Independent Testing

In [11]:
os.chdir("/home/t326h379/OGP")


df_negative = pd.read_csv('Feature_Extraction_O_linked_Training_Negative_114307_Sites_less.txt',header=None)

df_positive = pd.read_csv('Feature_Extraction_O_linked_Training_Positive_4885_Sites_less.txt',header=None)

Header_name = ["Position","PID","Position_redundant","81 Window sequence","S or T"]

col_of_feature = [i for i in range(1,1025)]

Header_name = Header_name + col_of_feature

df_positive.columns = Header_name
df_negative.columns = Header_name


frames = [df_positive, df_negative]

O_linked_training = pd.concat(frames,ignore_index = True)

df_Train_array = O_linked_training.drop(["Position","PID","Position_redundant","81 Window sequence","S or T"],axis=1)
df_Train_array = np.array(df_Train_array)

X_train_full = df_Train_array

y_train_full = np.array([1]*4885+[0]*114144)

print("Prior")
print(X_train_full.shape)
print(y_train_full.shape)
seed = 42
rus = RandomUnderSampler(random_state = seed)
X_train_full, y_train_full = rus.fit_resample(X_train_full,y_train_full)
print("After")
print(X_train_full.shape)
print(y_train_full.shape)

# Independent Test Dataset
df_negative_test = pd.read_csv('Feature_Extraction_O_linked_Testing_Negative_11466_Sites_less.txt',header=None)

df_positive_test = pd.read_csv('Feature_Extraction_O_linked_Testing_Positive_375_Sites_less.txt',header=None)

Header_name = ["Position","PID","Position_redundant","81 Window sequence","S or T"]

col_of_feature = [i for i in range(1,1025)]

Header_name = Header_name + col_of_feature

df_positive_test.columns = Header_name

df_negative_test.columns = Header_name


frames_test = [df_positive_test, df_negative_test]

O_linked_testing = pd.concat(frames_test,ignore_index = True)

df_Test_array = O_linked_testing.drop(["Position","PID","Position_redundant","81 Window sequence","S or T"],axis=1)
df_Test_array = np.array(df_Test_array)

X_test_full = df_Test_array

y_test_full = np.array([1]*374+[0]*11466)

# Training Starts From Here

from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

from imblearn.under_sampling import RandomUnderSampler

# Run the model for Three times and choose the best answer among them
a = random.sample(range(1, 1000000), 3)

for i in a:
    seed = i
    print("Seed : ", seed)

    rus = RandomUnderSampler(random_state = seed)
    X_train, y_train = rus.fit_resample(X_train_full,y_train_full)
    
    X_train =  X_train.reshape( X_train.shape[0],1024,1)

    x_train, x_val, y_train_1, y_val = train_test_split(X_train, y_train,random_state =21, test_size=0.1)

    y_train_1 = tf.keras.utils.to_categorical(y_train_1,2)
    y_val = tf.keras.utils.to_categorical(y_val,2)
    
    model = Sequential()

    model.add(Input(shape=(1024,1)))

    model.add(Conv1D(filters=64,kernel_size=3,activation='relu',name='Conv_1D_1_add'))
    model.add(MaxPooling1D(pool_size=2,name="MaxPooling1D"))
    model.add(Dropout(0.3))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(32,activation='relu',name="Dense_1"))
    model.add(Dropout(0.3))

    model.add(Dense(2,activation='softmax',name="Dense_2"))


    model.compile(optimizer=tf.keras.optimizers.Adam(),loss="binary_crossentropy",metrics=["accuracy"])

    checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath="ROC_ROC_Premise_Assumption.h5", 
                                    monitor = 'val_accuracy',
                                    verbose=0, 
                                    save_weights_only=False,
                                    save_best_only=True)

    reduce_lr_acc = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.001, patience=5, verbose=1, min_delta=1e-4, mode='max')

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=5,mode='max')

    history = model.fit(x_train, y_train_1,epochs=400,verbose=1,batch_size=256,
                            callbacks=[checkpointer,reduce_lr_acc, early_stopping],validation_data=(x_val, y_val))
    
    from imblearn.under_sampling import RandomUnderSampler
    
    from imblearn.under_sampling import RandomUnderSampler

    rus = RandomUnderSampler(random_state = seed)
    X_independent, y_independent = rus.fit_resample(X_test_full,y_test_full)
    
    X_independent = X_independent.reshape(X_independent.shape[0],1024,1)
    Y_pred = model.predict(X_independent)
    Y_pred = (Y_pred > 0.5)
    y_pred = [np.argmax(y, axis=None, out=None) for y in Y_pred]
    y_pred = np.array(y_pred)

    confusion = confusion_matrix(y_independent,y_pred)

    print("Matthews Correlation : ",matthews_corrcoef(y_independent, y_pred))
    print("Confusion Matrix : \n",confusion_matrix(y_independent, y_pred))
    print("Accuracy on test set:   ",accuracy_score(y_independent, y_pred))

    cm = confusion_matrix(y_independent, y_pred)

    TP = cm[1][1]
    TN = cm[0][0]
    FP = cm[0][1]
    FN = cm[1][0]

    mcc = matthews_corrcoef(y_independent, y_pred)

    Sensitivity = TP/(TP+FN)

    Specificity = TN/(TN+FP)

    print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)

    print(classification_report(y_independent, y_pred))

    fpr, tpr, _ = roc_curve(y_independent, y_pred)

    roc_auc_test = auc(fpr,tpr)



    print("Area Under Curve:   ",roc_auc_test)


Prior
(119029, 1024)
(119029,)
After
(9770, 1024)
(9770,)
Seed :  841264
Epoch 1/400
35/35 [==============================] - 1s 41ms/step - loss: 0.5951 - accuracy: 0.6817 - val_loss: 0.4825 - val_accuracy: 0.7810
Epoch 2/400
35/35 [==============================] - 1s 40ms/step - loss: 0.4885 - accuracy: 0.7797 - val_loss: 0.4402 - val_accuracy: 0.8025
Epoch 3/400
35/35 [==============================] - 1s 37ms/step - loss: 0.4471 - accuracy: 0.8072 - val_loss: 0.4055 - val_accuracy: 0.8137
Epoch 4/400
35/35 [==============================] - 1s 38ms/step - loss: 0.4067 - accuracy: 0.8323 - val_loss: 0.3918 - val_accuracy: 0.8229
Epoch 5/400
35/35 [==============================] - 1s 36ms/step - loss: 0.3825 - accuracy: 0.8402 - val_loss: 0.3795 - val_accuracy: 0.8311
Epoch 6/400
35/35 [==============================] - 0s 9ms/step - loss: 0.3601 - accuracy: 0.8527 - val_loss: 0.4096 - val_accuracy: 0.8086
Epoch 7/400
35/35 [==============================] - 0s 9ms/step - loss: 0.3